In [11]:
%%capture
%matplotlib inline
%time

!rm -rf tmp/*

# Had to install this funcy dependency manually, there was a bug in cadCAD, where the dependency wasn't loaded
# import sys
# !{sys.executable} -m pip install --user funcy

# instal line profiler, for optimizing notebook
# !pip install --user line_profiler
# !pip install --user Pillow

import os.path
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
import importlib
import policies
import config
import state_updates
import utils
from numpy.random import rand

importlib.reload(config)
importlib.reload(policies)
importlib.reload(state_updates)
importlib.reload(utils)

initial_conditions, disaster_ideal, rarity, market_settings = config.initial_conditions, config.disaster_ideal, config.rarity, config.market_settings

In [12]:
# height of each graph (for plotting)
plt_row_height = 7
plt_width = 20


params = {
    'bc-reserveRatio':  [0.75],       # bonding curve reserve ratio (connector weight)
    'bc-virtualBalance': [10],        # bonding curve virtual balance
    'bc-virtualSupply': [10000],      # bonding curve virtual supply
    'payMultiplier':    [0.327],      # used to calculate reward (multiplied by ratio of rarity)
    'priceMultiplier': [3],           # used to calculate cost to keep (mult by reward amt + base price)
    'basePrice': [3, 5]           # minimum price to keep any clover
}

# Set this to the params that you want to display when graphing a line-per-param-set
variableParams = ['basePrice']

previousRuns = (0) * 24
days = 1
timesteps_per_run = int(24 * days)
monte_carlo_runs = 1

from cadCAD.configuration.utils import config_sim
simulation_parameters = config_sim({
    'T': range(timesteps_per_run),
    'N': monte_carlo_runs,
    'M': params
})
simulation_parameters

[{'N': 1,
  'T': range(0, 24),
  'M': {'bc-reserveRatio': 0.75,
   'bc-virtualBalance': 10,
   'bc-virtualSupply': 10000,
   'payMultiplier': 0.327,
   'priceMultiplier': 3,
   'basePrice': 3}},
 {'N': 1,
  'T': range(0, 24),
  'M': {'bc-reserveRatio': 0.75,
   'bc-virtualBalance': 10,
   'bc-virtualSupply': 10000,
   'payMultiplier': 0.327,
   'priceMultiplier': 3,
   'basePrice': 5}}]

In [13]:
initialized_conditions = initial_conditions
# initialized_conditions = disaster_ideal
        

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, 
# the user specifies if state update functions will be run in series or in parallel
# and the policy functions that will be evaluated in that block
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            's': state_updates.initialize
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'participantPool': policies.participant_pool_policy
        },
        'variables': { # The following state variables will be updated simultaneously
            's': state_updates.update_participant_pool
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'minerPolicy': policies.miner_policy
        },
        'variables': { # The following state variables will be updated simultaneously
            's': state_updates.update_state_miner_policy
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'playerPolicy': policies.player_policy
        },
        'variables': { # The following state variables will be updated simultaneously
            's': state_updates.update_state
        }
    },
    { 
        'policies': { # The following policy functions will be evaluated and their returns will be passed to the state update functions
            'marketActivityPolicy': policies.market_activity_policy
        },
        'variables': { # The following state variables will be updated simultaneously
            's': state_updates.update_state
        }
    }
    #     speculators # folks speculating on clover coin price
]

In [14]:
from tabulate import tabulate
from cadCAD.configuration import append_configs
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# The configurations above are then packaged into a `Configuration` object
append_configs(
    initial_state=initial_conditions, #dict containing variable names and initial values
    partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
    sim_configs=simulation_parameters #dict containing simulation parameters
)

from cadCAD import configs
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

exec_mode = ExecutionMode()
multi_proc_ctx = ExecutionContext(context=exec_mode.multi_proc)
run = Executor(multi_proc_ctx, configs=configs)

In [15]:
i = 0
verbose = False
results = []
for raw_result, tensor_field in run.execute():
    if verbose:
        print()
        print(f"Tensor Field: {type(tensor_field)}")
        print(tabulate(tensor_field, headers='keys', tablefmt='psql'))
        print(f"Output: {type(result)}")
        print(tabulate(result, headers='keys', tablefmt='psql'))
        print()
    results.append({})
    results[i]['result'] = raw_result
    results[i]['simulation_parameters'] = simulation_parameters[i]
    i += 1

multi_proc: [<cadCAD.configuration.Configuration object at 0x7f621cbe4f98>, <cadCAD.configuration.Configuration object at 0x7f621cbe44a8>, <cadCAD.configuration.Configuration object at 0x7f621cbe4cf8>, <cadCAD.configuration.Configuration object at 0x7f621cbe4e48>, <cadCAD.configuration.Configuration object at 0x7f61e49d5c88>, <cadCAD.configuration.Configuration object at 0x7f61e49d5dd8>]
[<cadCAD.configuration.Configuration object at 0x7f621cbe4f98>, <cadCAD.configuration.Configuration object at 0x7f621cbe44a8>, <cadCAD.configuration.Configuration object at 0x7f621cbe4cf8>, <cadCAD.configuration.Configuration object at 0x7f621cbe4e48>, <cadCAD.configuration.Configuration object at 0x7f61e49d5c88>, <cadCAD.configuration.Configuration object at 0x7f61e49d5dd8>]


TypeError: 'int' object is not subscriptable

In [ ]:
import graphing
importlib.reload(graphing)

graphing.make_title_bar(results, graphsize=(10,5))
graphing.make_graph(results, graphing.clovers_metrics_graph, graphsize=(10,5))
graphing.make_graph(results, graphing.clovers_traded_graph, graphsize=(10,5))
graphing.make_graph(results, graphing.bc_balance_graph, graphsize=(10,5))

In [ ]:
graphing.make_title_bar(results, graphsize=(10,5))
graphing.make_graph(results, graphing.num_syms_graph, graphsize=(10,5))
graphing.make_graph(results, graphing.rewards_per_sym_graph, graphsize=(10,5))
graphing.make_graph(results, graphing.rewards_per_sym_eth_graph, graphsize=(10,5))

In [ ]:
importlib.reload(graphing)
graphing.make_title_bar(results, graphsize=(10,5))
graphing.make_final_state_graph(results, graphsize=(10,6))

In [ ]:
variableParams = ['priceMultiplier']

graphing.make_param_runs_graph(results, variableParams, ['numPlayerClovers','numBankClovers', 'slope'])

In [ ]:
graphing.make_param_runs_graph(results, variableParams, ['bc-balance', 'bc-totalSupply'])